<span class='note'>*Make me look good.* Click on the cell below and press <kbd>Ctrl</kbd>-<kbd>Enter</kbd>.</span>

In [ ]:
from IPython.core.display import HTML
def css_styling():
    styles = open('css/custom.css', 'r').read()
    return HTML(styles)
css_styling()

<h5 class='prehead'>SA367 &middot; Mathematical Models for Decision Making &middot; Spring 2017 &middot; Uhan</h5>

<h5 class='lesson'>Lesson 6.</h5>

<h1 class='lesson_title'>The Mileage Running Problem</h1>

## The problem

Professor May B. Wright needs to fly from Baltimore (BWI) to Los Angeles (LAX) to attend a conference.
She thinks this would be the perfect opportunity to accumulate some frequent flyer miles on American Airlines (AA), where she already has Platinum status.

Looking into flights on AA, she sees that every itinerary from BWI to LAX costs roughly the same.
She has a full day to spare for travel, so she wants to know: which sequence of AA domestic flights starting at BWI and ending at LAX over the course of one day will allow her to accumulate the most miles?

* Yes, people actually do this. This is known as __mileage running__. 
    - Apparently, this has become harder to do in recent years.
    - [A recent article from the New York Times](https://www.nytimes.com/2014/09/14/upshot/the-fadeout-of-the-mileage-run.html).
    - [An older article from Wired](https://www.wired.com/2007/07/mileage-runner/).

## Modeling the problem

* Suppose we have a database of every AA domestic flight on a given day.

* In particular, for each flight, we have:
    - the flight number
    - the origin airport
    - the destination airport
    - the departure time at the origin airport
    - the arrival time at the destination airport
    - the distance traveled in miles

* How can we formulate Professor Wright's problem as a shortest path problem?

## pandas (the package, not the animals)

* In the same folder as this notebook, there is a file called `aa_domestic_flights.csv` with the database described above.

* `.csv` stands for **comma-separated values**.

* We can view `.csv` files in Excel - let's see what's in this file. _Cut to Excel..._

* How can we use this data in Python? With __`pandas`__.

* `pandas` is a Python package for data analysis. 
    - It's especially useful for cleaning, merging, and manipulating datasets.

* `pandas` does a lot of stuff &mdash; here are a few resources:
    - [Here is the documentation for `pandas`](http://pandas.pydata.org/pandas-docs/stable/index.html).
    - [Chris Albon's notes](http://chrisalbon.com) are also a good resource on how to get things done with `pandas` (look in the _Data Wrangling_ section). 
    
* In this lesson, we'll use `pandas` in a very basic way to help us set up the shortest path problem we formulated above.

* To install `pandas`, open a <span class="rred">WinPython Command Prompt</span> and type

```
pip install pandas
```

* `pip` might tell you that `pandas` is already installed. If not, it should go ahead and install it for you.

* To use `pandas`, we first need to import it, like this:

In [ ]:
import pandas as pd

* A `pandas` __DataFrame__ is just a two-dimensional table, with rows and columns.

* The code below uses the __`read_csv()`__ function in `pandas` to read `aa_domestic_flights.csv` into a DataFrame called `df`.
    - [Documentation for `read_csv()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html)

In [ ]:
df = pd.read_csv('aa_domestic_flights.csv', parse_dates=['DEP_TIME', 'ARR_TIME'])

* By default, `read_csv()` assumes the names of each column are in the first row.

* The `parse_dates` argument above tells `read_csv()` which columns correspond to dates, so that we can perform date-specific calculations on these columns later.

* It's a good idea to take a quick look at the DataFrame `read_csv()` creates, just in case something went wrong.

* To examine the first 5 rows of a DataFrame, we can use the `.head()` method:

In [ ]:
# Print the first 5 rows of df
df.head()

* Another useful methods is `.describe()`. 

* By default, `.describe()` only provides summary statistics for the columns with numeric data. 

* To get summary statistics on all the columns, we need to include the argument `include="all"`, like this:

In [ ]:
df.describe(include="all")

* A column by itself is called a **Series**.

* You can select the `DEST` column or Series of the DataFrame `df` like this:

```
df["DEST"]
```

* So, to print the `DEST` Series, we could write:

In [ ]:
# Print the DEP_TIME column
print(df["DEST"])

## Setting up the shortest path problem in networkx

* Now that we can access the flight database in Python, we can use its contents to setup the shortest path problem we formulated above.

* First, let's import `networkx` and `bellmanford` so we can use them:

In [ ]:
import networkx as nx
import bellmanford as bf

### Getting a list of flights and airports

* It will be useful to create a list called `flights` containing of the flights.

* What part of the dataset contains this information?

_Write your notes here. Double-click to edit._

* From the `.describe()` output above, we see that the flights in `df["FLIGHT"]` are unique.

* We can can convert the Series `df["FLIGHT"]` to a list with the function `list()`. Then we can use the list methods we learned about earlier, such as `.append()`, if necessary.

In [ ]:
# Take the FLIGHT column from df, convert to a list
flights = list(df["FLIGHT"])

* Let's inspect the variable `flights` we just created:

In [ ]:
# Print flights
print("Flights: {0}".format(flights))

* You might want to click on the left of the output above &mdash; this will collapse the output so it doesn't take over your browser window.

* It's a good idea to make sure nothing funny happened - let's make sure we have the right number of flights in the variable `flights`:

In [ ]:
print("Number of flights: {0}".format(len(flights)))

* Let's also create a list of all the airports.

* What part of the dataset contains this information?

_Write your notes here. Double-click to edit._

* To be absolutely sure we have all the airports in the data set, let's create a variable `airports` like this:

In [ ]:
# Eliminate all duplicates of the ORIGIN and DEST columns from df, 
# take their union, convert to a list
airports = list(set(df["ORIGIN"]) | set(df["DEST"]))

* Roughly speaking, this is what the above code does:
    - `set(df["ORIGIN"])` eliminates all duplicates in `df["ORIGIN"]` and returns a list-like object called a __set__.
    - Same goes for `set(df["DEST"])`.
    - The `|` operator takes the __union__ of the sets `set(df["ORIGIN"])` and `set(df["DEST"])`. This is almost what we want, but...
    - Sets are similar to lists, but have their own methods. We can turn the set into a list with the function `list()`.

* Let's print out the list of airports and the number of airports to make sure everything looks OK:

In [ ]:
print('Airports: {0}'.format(airports))
print('Number of airports: {0}'.format(len(airports)))

### Adding nodes with attributes

* Now we're ready to build the shortest path graph. Let's start with an empty directed graph:

In [ ]:
# Create empty digraph
G = nx.DiGraph()

* Next, let's create a "start" and "end" node.

In [ ]:
# Create start and end nodes
G.add_node("start")
G.add_node("end")

* Now, we need to add a node for each flight, or each row of our database.

* We can quickly iterate through the rows of a DataFrame using the `.itertuples()` method:

```python
for row in df.itertuples()
    # Put some code here
    # row.COLUMN_NAME is how you would get the value of 
    # the column COLUMN_NAME in the current row
```

* So we can add a node for each flight like this:

In [ ]:
# Add a row for each flight
for row in df.itertuples():
    G.add_node(row.FLIGHT, origin=row.ORIGIN, dest=row.DEST, dep_time=row.DEP_TIME, arr_time=row.ARR_TIME, distance=row.DISTANCE)

* Wait &mdash;
```python
G.add_node(row.FLIGHT)
```
adds a node whose name is the value of `row.FLIGHT`. What is all the other stuff?

* Remember in the last lesson when we added the "length" attribute to each edge? Like this?

```python
G.add_edge(1, 2, length=9)
```
* We can add attributes to nodes as well. This will come in useful later.

* To access a particular attribute of a node, we write something like this:

In [ ]:
# Find the departure time of flight "1-BOS-JFK"
G.node["1-BOS-JFK"]["dep_time"]

* Note that the departure time is recorded as a __Timestamp__ &mdash; a special `pandas` data structure just fo rdates and times.

* To add/subtract from Timestamps, we'll need to use special functions, as we'll see shortly.

* This might seem awkward, but if you think about it, working with dates and time is a pain &mdash; you need to keep track of different (non-base-10) units.

* The `.nodes()` method applied to a `networkx` graph returns a list of its nodes. 
    - If we input `data=True` into `.nodes()`, it will return the node attribute information as well.

* So we can check our work by printing out all the nodes in `G` like this:

In [ ]:
# Print all of the nodes in G
print(G.nodes(data=True))

* The `.number_of_nodes()` method applied to a `networkx` graph &mdash; well, you can guess what it does. Or, you can just try it out:

In [ ]:
# Print number of nodes in G
print(G.number_of_nodes())

### Adding edges

* Now we can check every pair of flight nodes, and check if we need to add an edge between them.
    - Remember the length of these edges is the negative of the distance of the first flight.
    - To add or subtract times, we need to use `pd.to_timedelta()` &mdash; [here is the documentation](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_timedelta.html).
    - For example, to subtract 30 minutes, we would write 
    ```python
    some_time_variable - pd.to_timedelta(30, unit="m")
    ```

In [ ]:
# Iterate through every pair of flight nodes
for first in flights:
    for second in flights:
        
        # If the first flight arrives where the second flight departs...
        if (G.node[first]["dest"] == G.node[second]["origin"]):
            
            # If the first flight arrive 45 minutes before the second flight leaves...
            if (G.node[first]["arr_time"] + pd.to_timedelta(45, unit="m") < G.node[second]["dep_time"]):
                G.add_edge(first, second, length=-G.node[first]["distance"])

* Finally, we need to add edges of length 0:
    - from the start node to all flights departing from BWI, and
    - from all flights arriving at LAX to the end node.

In [ ]:
# Iterate through all flights
for flight in flights:

    # If the flight departs from BWI...
    if G.node[flight]["origin"] == "BWI":
        G.add_edge("start", flight, length=0)
        
    # If the flight arrives at LAX...
    if G.node[flight]["dest"] == "LAX":
        G.add_edge(flight, "end", length=0)

* Similar to `G.nodes()` and `G.number_of_nodes()`, we can check our work with `G.edges()` and `G.number_of_edges()`.

In [ ]:
# Print all the edges in G, with attribute information
G.edges(data=True)

In [ ]:
# Print the number of edges in G
G.number_of_edges()

## Solving the shortest path problem, interpreting the output

* Now that we have our directed graph set up, we can solve for the shortest path from the start node to the end node just like we did in the last lesson:

In [ ]:
length, nodes, negative_cycle = bf.bellman_ford(G, source="start", target="end", weight="length")

print("Negative cycle? {0}".format(negative_cycle))
print("Shortest path length: {0}".format(length))
print("Shortest path: {0}".format(nodes))

* What does the output tell us about Professor Wright's problem?

_Write your notes here. Double-click to edit._